In [52]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import lxml.html as lh
import re
import datetime

In [42]:
URL = "https://sportsbook.draftkings.com/leagues/baseball/88670847?category=game-lines&subcategory=game"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [107]:
#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

rows = [row.text_content() for row in tr_elements]

In [108]:
rows[18]

'12:05AMHOU AstrosJake Odorizzi-1.5-125O\xa09-110-200'

In [109]:
today_index = rows.index('Today RUN LINETOTALMONEYLINE')
tomorrow_index = rows.index('Tomorrow RUN LINETOTALMONEYLINE')

tomorrow_index

17

In [110]:
def time_adj(time_str, timedelta):
    
    time = datetime.datetime.strptime(time_str, '%I:%M%p').time()
    
    start = datetime.datetime(
        2000, 1, 1,
        hour=time.hour, minute=time.minute, second=time.second)
    end = start + timedelta
    
    return_time = end.time().strftime('%I:%M%p')
    return return_time

In [170]:
## Using regular expressions, we find the draft kings header for 2 days from now

exp = "[A-Z]{3}\ [A-Z]{3}\ "
r = re.compile(exp)

final_divider = rows.index(list(filter(r.match, rows))[0])

final_divider

58

In [189]:
test_rows = rows.copy()
test_rows.remove("Tomorrow RUN LINETOTALMONEYLINE")

rows_for_today = []

max_time = datetime.datetime.strptime("20/01/2020" ,"%d/%m/%Y").time()

for row in test_rows[1:57]:
        
    time_stamp = re.search("[0-9]{1,2}\:[0-9]{1,2}[A-Z]{2}", row).group()
    time_stamp_adj = time_adj(time_stamp, datetime.timedelta(hours = -4))
    time = datetime.datetime.strptime(time_stamp_adj, '%I:%M%p').time()
    
    if time >= max_time:
        
        max_time = time
        
        rows_for_today.append(row)
        
    else:
        break


In [190]:
rows_for_today

['11:05PMTB RaysShane McClanahan-1.5-155O\xa010-115-240',
 '11:05PMBAL OriolesMatt Harvey+1.5+135U\xa010-105+195',
 '11:05PMARI DiamondbacksTaylor Widener+1.5+105O\xa09+100+210',
 '11:05PMPHI PhilliesAaron Nola-1.5-125U\xa09-120-260',
 '11:05PMSTL CardinalsJ.A. Happ-1.5+105O\xa09+100-155',
 '11:05PMPIT PiratesDillon Peters+1.5-125U\xa09-120+135',
 '11:10PMCIN RedsWade Miley-1.5+115O\xa08-110-140',
 '11:10PMMIA MarlinsZach Thompson+1.5-135U\xa08-110+120',
 '11:10PMTOR Blue JaysSteven Matz-1.5-115O\xa09.5-115-165',
 '11:10PMDET TigersMatt Manning+1.5-105U\xa09.5-105+145',
 '11:10PMBOS Red SoxEduardo Rodríguez-1.5-130O\xa010-110-195',
 '11:10PMCLE IndiansLogan Allen+1.5+110U\xa010-110+165',
 '11:10PMWAS NationalsPaolo Espino+1.5-120O\xa09+100+170',
 '11:10PMNY MetsRich Hill-1.5+100U\xa09-120-200',
 '11:20PMSF GiantsKevin Gausman+1.5-180O\xa08.5-110+115',
 '11:20PMATL BravesMax Fried-1.5+155U\xa08.5-110-135',
 '12:05AMHOU AstrosJake Odorizzi-1.5-125O\xa09-110-200',
 '12:05AMTEX RangersGlen

In [248]:
game_moneylines = []

for i in range(0,len(rows_for_today),2):
    
    away_team_row = rows_for_today[i]
    home_team_row = rows_for_today[i + 1] 
    
    ## We get todays date
    date = datetime.datetime.today().strftime('%m/%d')
    
    ## This regex method is a little clunky, but it works with one word and two word teams given Draft kings formatting
    ## We get the away teams name
    away_team_list = re.findall(" [A-Z][a-z]+", away_team_row)
    del(away_team_list[-1])
    away_team = ''.join(away_team_list)[1:]
    
    ## and the home tams name
    home_team_list = re.findall(" [A-Z][a-z]+", home_team_row)
    del(home_team_list[-1])
    home_team = ''.join(home_team_list)[1:]
    
    ## We get the away teams moneyline
    away_team_moneyline = re.findall("((\+|\-)[0-9]+)", away_team_row)[-1][0]
    
    ## We get the home teams moneyline
    home_team_moneyline = re.findall("((\+|\-)[0-9]+)", home_team_row)[-1][0]
    
    moneyline_row = [date, away_team, home_team, away_team_moneyline, home_team_moneyline]
    

    game_moneylines.append(moneyline_row)

game_moneylines_df = pd.DataFrame(game_moneylines, columns=['Date', 'away_team', 'home_team','away_team_moneyline','home_team_moneyline'])

game_moneylines_df.head()

,Date,away_team,home_team,away_team_moneyline,home_team_moneyline
0,08/27,Rays,Orioles,-240,+195
1,08/27,Diamondbacks,Phillies,+210,-260
2,08/27,Cardinals,Pirates,-155,+135
3,08/27,Reds,Marlins,-140,+120
4,08/27,Blue Jays,Tigers,-165,+145


In [249]:
game_moneylines_df.to_csv("dk_scrape_output.csv", index=False)